In [ ]:
from collections import namedtuple
import json
from pathlib import Path
import panel as pn

## Load config:
config_path = Path('config.json')
if not config_path.is_file():
    !python update_config.py
if config_path.is_file():
    with open(config_path,encoding='utf-8') as config_file:
        config = json.load(config_file)
else:
    raise ValueError("config.json doesn't exist!")

import dask
from dask.diagnostics import ProgressBar
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.xarray
from io import StringIO
import numpy as np
import pandas as pd
from panel.viewable import Viewer
from bokeh.models import HoverTool
import param
from shapely.geometry import Point, LineString
import xarray as xr
import xclim as xc
from xclim import analog as xa
from datetime import datetime
import pickle
import joblib
import warnings
from shapely.errors import ShapelyDeprecationWarning
from core import utils,compress,search


In [ ]:
dask.config.set(scheduler=config["options"]["dask_schedule"], temporary_directory='/notebook_dir/writable-workspace/tmp')
try:
    curr_dir = Path(__file__).parent
except NameError:  # When running as a notebook "__file__" isn't defined.
    curr_dir = Path('.')
cities_file = f'{curr_dir}/cities_tmp.geojson'

# Projection
biasadjust = config["options"]["biasadjust"] # scaling or dqm the method used for the annual adjustment method

# Random city on load
init_rand_city = config["options"]["init_rand_city"]

# Data
dref = utils.open_thredds(
    config["url"]["dref"]
).chunk({'time': -1}).sel(time=slice('1991', '2020'))

dsim = utils.open_thredds(config["url"]["dsim"])
dsim = xc.core.calendar.convert_calendar(dsim, 'default')

places = gpd.read_file(config["url"]["places"]).to_crs(epsg=8858)

cities = gpd.read_file(cities_file)

# Pre-compute/download the reference distributions and the density map
if not Path('benchmark.obj').is_file():
    benchmark = utils.open_thredds(config["url"]["benchmark"]).benchmark.load()

    with open('benchmark.obj', 'wb') as obj_handler:
        pickle.dump(benchmark, obj_handler)
else:
    with open('benchmark.obj', 'rb') as obj_handler:
        benchmark = pickle.load(obj_handler)

if not Path('density.obj').is_file():
    masks = utils.open_thredds(config["url"]["masks"])

    density = masks.dens_adj.sel(year=2020).where(
        masks.roi & dref.isel(time=0).notnull().to_array().all('variable')
    ).load()

    with open('density.obj', 'wb') as obj_handler:
        pickle.dump(density, obj_handler)

# load pickled data
else:
    with open('density.obj', 'rb') as obj_handler:
        density = pickle.load(obj_handler)

In [ ]:
best_analog_mode = 'closestPer'  # closestN or min, see below
analog_modes = ['closestPer','closestN','min']
icity = 0
city = cities.iloc[icity]
climate_indices = ['TNn']
all_indices = [x.name for k,x in dsim.data_vars.items()]
tgt_period = slice("2041", "2070")
opts = {{"en":f"{x-29}-{x}","fr":f"{x-29} à {x}"}["en"]: slice(f"{x-29}", f"{x}") for x in range(2020, 2101, 10)}
periods = list(opts.values())
ssp = 'ssp245'
ssp_list = dsim.ssp.values
num_realizations = 12
max_real = 12
density_factor = 4
max_density = 10

In [ ]:
ana = search.analogs( dsim,
             dref,
             density,
             benchmark,
             city,cities,places,
             climate_indices,
             density_factor,max_density,
             tgt_period,periods,
             ssp,ssp_list,
             best_analog_mode=best_analog_mode, analog_modes=analog_modes,
             num_realizations=num_realizations, max_real=max_real)

In [ ]:
sim = ana[1]
ref = ana[2]
sim_tgt = sim.sel(time=tgt_period).drop_vars(['lon', 'lat'])
dissimilarity = xa.spatial_analogs(
    sim_tgt, ref, dist_dim='time', method='zech_aslan'
)
simzscore = xa.spatial_analogs(
    sim_tgt.mean('realization'), sim_tgt, dist_dim='time', method='seuclidean'
)
percs = utils.get_score_percentile(dissimilarity, climate_indices, benchmark)
ilat_ref = density.indexes["lat"]
ilon_ref = density.indexes["lon"]
utils.inplace_compute(sim, ref)
dissimilarity, percs, simzscore = dask.compute(dissimilarity, percs, simzscore)

diss = dissimilarity.isel(realization=0)
dists = utils.get_distances(diss.lon, diss.lat, city.geometry)

In [ ]:
utils.get_distances([city.lon_raw],[city.lat_raw],city.geometry)

In [ ]:
df = gpd.GeoDataFrame(geometry=[Point(city.lon_raw,city.lat_raw)], crs=4326)
df.to_crs(epsg=8858).distance(gpd.GeoDataFrame(geometry=[city.geometry], crs=4326).to_crs(epsg=8858))

In [ ]:
gpd.GeoDataFrame(geometry=[city.geometry], crs=4326)

In [ ]:
city.lat_raw

In [ ]:
import base64

In [ ]:
base64.b85encode(bytes(bytelistB))

In [ ]:
import time
t0 = time.time()
import clisops.subset as cc
t1 = time.time()
print(t0 - t1)

In [ ]:
from clisops.core.subset import distance